![](../images/nci-logo.png)
-------
# Data access, processing and manipulation on NCI's VDI and Raijin

## Magnetotellurics (MT) data example



### In this notebook:

- abcdefg within the VDI
    - <a href='#part1'>Accessing MT data from NCI</a>  
    - <a href='#part2'>Time-series QC using snuffler, a miniSEED time-series visualisation tool</a>  
    - <a href='#part3'>Running the Bounded Influence, Remote Reference Processing (BIRRP) code on the VDI and Raijin</a> 
    - <a href='#part4'>Using MTpy to convert BIRRP outputs to EDI files </a> 
    - <a href='#part5'>Converting BIRRP outputs to NetCDF4</a>
    - <a href='#part6'>Running Occam2D and ModEM on Raijin</a>
    - <a href='#part7'></a>

   
#### The following material uses The University of Adelaide Magnetotellurics Data Collection which is available under the Create Commons License 4.0.  

---------

<br>


<a id='part1'></a> 
# Accessing MT data from NCI 

## NCI Data Catalogue

For this example, we will access some of The University of Adelaide MT Data. To search for this, we can fisit the NCI Data Catalogue ( https://datacatalogue.nci.org.au/ ).

<img src="./images/NCI_data_catalogue_1.png">

<br>


### Let's begin by searching for magnetotelluric datasets available at NCI

<img src="./images/NCI_data_catalogue_2.png">

<br>

### Now click on any dataset of interest.

<img src="./images/NCI_data_catalogue_3.png">

<br>

### This will bring you to the dashboard page 

<img src="./images/NCI_data_catalogue_4.png">

<br>

### This page has a description of the dataset as well as links to the data on THREDDS, Geonetwork and a file path on Raijin.

<img src="./images/NCI_data_catalogue_4b.png">

<br>

### Now let's click on the Geonetwork link

<img src="./images/NCI_data_catalogue_4c.png">

<br>

# NCI Geonetwork


<img src="./images/NCI_data_catalogue_5.png">

<br>


### At the top of the Geonetwork page we can see a short description of the dataset. Below this description we can see links to the THREDDS data server (TDS) as well as links to any associated publications that have used this dataset.

<img src="./images/NCI_data_catalogue_5b.png">

<br>

<a id='part2'></a> 
## Time-series QC using miniSEED visualisation tools (e.g. snuffler)

For this example, we need to first log onto NCI's VDI. Once on the VDI, open up a terminal and load in the following modules:

```
    $ module load python/2.7.11
    $ module load ipython/4.2.0-py2.7
    $ module load scipy/0.17.1-py2.7
    $ module load matplotlib/1.5.1-py2.7
    $ module load virtualenv/py2.7
    $ module load pyqt/4.11.4-qt-4.8.7-py2.7
```    

Next let's set up a virtual environment to work in:

```
    $ virtualenv mseed_venv
    $ source mseed_venv/bin/activate

```    

Now we need to install Pyrocko ( https://pyrocko.org/ ) in our virtual environment. For those who aren't familiar, Pyrocko is an open source seismology toolbox and library, written in the Python programming language. It can be utilized flexibly for a variety of geophysical tasks, like seismological data processing and analysis, modelling of InSAR, GPS data and dynamic waveforms, or for seismic source characterization. 

Development and support is coordinated at https://github.com/pyrocko


```
    $ git clone git://github.com/pyrocko/pyrocko.git pyrocko
    $ cd pyrocko
    $ python setup.py install

```    

Once pyrocko is installed, we will use the Snuffler seismogram browser and workbench. The Snuffler is an interactive and extensible seismogram browser (but can also be used to view MT time-series) that is suited for small and very big datasets and archives.

For this example, let's view some miniSEED data located here:


```
    $ cd /g/data/my80/States_and_Territories/test/TS/example_TS_capricorn/
    
``` 
Now to view the example time series:

```
    $ snuffler *.mseed
``` 
and a window will open
<img src="./images/Snuffler1.png">

Initially there are no waveforms. To zoom in and view the waveforms, you need to hold the mouse button on the coloured boxes and pull towards you to zoom in. Once you zoom in close enough, the time-series will appear. To zoom out, you hold the mouse over the trace data and push the mouse away from you.

<img src="./images/Snuffler2.png">

Navigation commands:

< space > - move forward one page in time

b - move backward one page in time

f - full screen is displayed

< minus > - remove one trace from the snuffler screen

< plus > - add a trace to the snuffler screen

: - toggle the snuffler command line

? - displays help information

For a more detailed tutorial on how to use snuffler, please visit https://pyrocko.org/docs/current/apps/snuffler/tutorial.html




<a id='part3'></a> 
## Running the Bounded Influence, Remote Reference Processing (BIRRP) code on the VDI and Raijin

Once we are happy with the quality of our time-series, the next step is to run magnetotelluric data processing. For this example, we will run the Bounded Influence, Remote Reference Processing (BIRRP) code of A.D. Chave and D.J. Thomsom. 

For those unfamiliar, the BIRRP program computes magnetotelluric and geomagnetic depth sounding response functions using a bounded influence, remote reference method, along with an implementation of the jackknife to get error estimates on the result. It incorporates a method for controlling leverage points (i.e., magnetic field values which are statistically anomalous), includes the implementation of two stage processing which enables removal of outliers in both the local electric and magnetic field variables, and allows multiple remote reference sites to be used. 

For more information about BIRRP and links to relevant literature, please visit http://www.whoi.edu/science/AOPE/people/achave/Site/Next1.html


###
The conditions for the use of BIRRP are:

 1) The robust bounded influence magnetotelluric analysis program, hereinafter called BIRRP, is provided on a caveat emptor basis. The author of BIRRP is not responsible for or culpable in the event of errors in processing or interpretation resulting from use of this code.

 2) No payment will be accepted by any user for data processing with BIRRP.

 3) BIRRP will not be distributed to anyone. Interested persons should be referred to this website.

 4) The author will be notified of any possible coding errors that are encountered.

 5) The author will be informed of any improvements or additions that are made to BIRRP.

 6) The use of BIRRP will be acknowledged in any publications and presentations that ensue. 



If these conditions are acceptable, send e-mail to achave@whoi.edu. The body of the message should state "I accept the conditions under which BIRRP is distributed" and copy the above six conditions. A gzipped tar file containing the source code will be distributed by return e-mail.

Once you have tge gzipped tar file, copy it over from your local machine to either raijin or the VDI:

```
    $ rsync -zvr < BIRRP tar file > abc123@raijin.nci.org.au:< raijin directory {e.g. /short/....} >
    
    or
    
    $ rsync -zvr < BIRRP tar file > abc123@vdi-< vdi number {e.g. n28} >.nci.org.au:< vdi directory {e.g. /local/....} >
    
    where abc123 is your NCI username.
``` 

Next let's untar the BIRRP code:

```
    $ tar -zxvf < BIRRP tar file >
``` 


Now we need to create a "Makefile" to be used to compile the Fortran 77 BIRRP code. Open up your favourite text editor (e.g. Vi, Nano, emacs) and create the following Makefile:

<pre>
FC = ifort

FCFLAGS = -O0 -debug -traceback -check -ftrapuv

TARGETS = clean birrp

OBJSOC = diagnostic.f math.f rtpss.f zlinpack.f coherence.f fft.f rarfilt.f utils.f dataft.f filter.f response.f weight.f  birrp.f

all:  $(TARGETS)

clean:$
        rm -f *.o *.mod birrp

birrp:$(OBJSOC)
        $(FC) $(FCFLAGS) -o $@ $(OBJSOC)

.SUFFIXES: .f .o

.f .o:
        $(FC) $(FCFLAGS) -c -o $@ $<

</pre>


Save this as Makefile. Before we compile the BIRRP code, we need to load in the intel-fc module:

```
    $ module load intel-fc
``` 

Now we can compile the code by typing

```
    $ make
``` 

You should now have the executable **birrp**. 


Let's now test our birrp code out on some example time-series. 




